In [1]:
test_input = """R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)"""

test_input_mine = """R 12 (#70c710)
U 6 (#0dc571)
L 4 (#5713f0)
D 3 (#d2c081)
L 4 (#59c680)
U 3 (#411b91)
L 4 (#8ceee2)
D 6 (#caa173)"""

In [2]:
input = open("inputs/18").read()

In [3]:
# directions = {
#     "R": (0, 1),
#     "L": (0, -1),
#     "U": (-1, 0),
#     "D": (1, 0)
# }

# lord forgive me im leaving np convention and using math convention
directions = {
    "R": (1, 0),
    "L": (-1, 0),
    "U": (0, 1),
    "D": (0, -1)
}

In [4]:
def add_tuples(tuple1, tuple2):
    return tuple(map(lambda x, y: x + y, tuple1, tuple2))

def mult_tuple(tpl, c):
    return tuple(c*x for x in tpl)

In [5]:
def p1(input, start=(1, -1)):
    pos = (0, 0)
    dug = [pos]

    for l in input.splitlines():
        dir, num_str, _ = l.split()
        num = int(num_str)

        for i in range(num):
            pos = add_tuples(pos, directions[dir])
            dug.append(pos)

    assert dug[0] == (0,0) and dug[-1] == (0, 0)
    dug = dug[:-1]

    dug_set = set(dug)

    filled = set()

    to_visit = [start]

    while to_visit:
        pos = to_visit.pop()

        # hit edge or already filled
        if pos in dug_set or pos in filled:
            continue
        
        filled.add(pos)

        to_visit.append((add_tuples(pos, directions['D'])))
        to_visit.append((add_tuples(pos, directions['U'])))
        to_visit.append((add_tuples(pos, directions['L'])))
        to_visit.append((add_tuples(pos, directions['R'])))
    
    print(len(filled), len(dug_set))
    
    return len(filled) + len(dug_set)

In [6]:
p1(test_input), p1(input)

24 38
44869 3634


(62, 48503)

In [7]:
p1(test_input_mine, start=(1, 1))

40 42


82

In [8]:
test_hex = "#70c710"

In [9]:
dirs = ['R', 'D', 'L', 'U']

num = int(test_hex[1:-1], base=16)
dir = dirs[int(test_hex[-1])]

dir, num

('R', 461937)

In [10]:
sm = 0
for l in test_input.splitlines():
# for l in input.splitlines():
    _, _, hex_str = l.split()

    dir = dirs[int(hex_str[-2])]
    num = int(hex_str[2:-2], base=16)

    # print(dir, num)
    sm += num

sm

6405262

In [11]:
pos = (0, 0)

horizontal_segments = []
vertical_segments = []

# for l in test_input_mine.splitlines():
for l in test_input.splitlines():
    dir, num_str, _ = l.split()
    num = int(num_str)

    print(dir, num)
    next_pos = add_tuples(pos, mult_tuple(directions[dir], num))

    if dir in ['R', 'L']:
        horizontal_segments.append((pos[1], tuple(sorted((pos[0], next_pos[0])))))
    else:
        vertical_segments.append((pos[0], tuple(sorted((pos[1], next_pos[1])))))

    pos = next_pos

R 6
D 5
L 2
D 2
R 2
D 2
L 5
U 2
L 1
U 2
R 2
U 3
L 2
U 2


In [12]:
horizontal_segments, vertical_segments

([(0, (0, 6)),
  (-5, (4, 6)),
  (-7, (4, 6)),
  (-9, (1, 6)),
  (-7, (0, 1)),
  (-5, (0, 2)),
  (-2, (0, 2))],
 [(6, (-5, 0)),
  (4, (-7, -5)),
  (6, (-9, -7)),
  (1, (-9, -7)),
  (0, (-7, -5)),
  (2, (-5, -2)),
  (0, (-2, 0))])

In [13]:
horizontal_lines = sorted(list(set([x[0] for x in horizontal_segments])))
vertical_lines = sorted(list(set([x[0] for x in vertical_segments])))

In [14]:
horizontal_lines, vertical_lines

([-9, -7, -5, -2, 0], [0, 1, 2, 4, 6])

In [15]:
import itertools

vertical_sides = []
horizontal_sides = []

vertex_points = []
interior_rectangle = []

for ((x_start, x_end), (y_start, y_end)) in itertools.product(itertools.pairwise(vertical_lines), itertools.pairwise(horizontal_lines)):
    # interior area
    # interior_area = (x_end - (x_start+1)) * (y_end - (y_start+1))

    v1 = (x_start, y_start)
    v2 = (x_end, y_start)

    v3 = (x_start, y_end)
    v4 = (x_end, y_end)

    vertex_points.extend([v1, v2, v3, v4])
    x_extent = x_end - x_start > 1
    y_extent = y_end - y_start > 1

    # keep track of all four sides, but only storing the interior of the sides
    # use the same format for horizontal and vertical sides that I did above

    if x_extent:
        horizontal_sides.append((y_start, (x_start+1, x_end-1)))
        horizontal_sides.append((y_end, (x_start+1, x_end-1)))

    if y_extent:
        vertical_sides.append((x_start, (y_start+1, y_end-1)))
        vertical_sides.append((x_end, (y_start+1, y_end-1)))

    # now keep the interior of the rectangle
    if x_extent and y_extent:
        interior_rectangle.append((x_start+1, x_end-1, y_start+1, y_end-1))

In [16]:
interior_rectangle

[(3, 3, -8, -8),
 (3, 3, -6, -6),
 (3, 3, -4, -3),
 (3, 3, -1, -1),
 (5, 5, -8, -8),
 (5, 5, -6, -6),
 (5, 5, -4, -3),
 (5, 5, -1, -1)]

In [17]:
horizontal_sides

[(-9, (3, 3)),
 (-7, (3, 3)),
 (-7, (3, 3)),
 (-5, (3, 3)),
 (-5, (3, 3)),
 (-2, (3, 3)),
 (-2, (3, 3)),
 (0, (3, 3)),
 (-9, (5, 5)),
 (-7, (5, 5)),
 (-7, (5, 5)),
 (-5, (5, 5)),
 (-5, (5, 5)),
 (-2, (5, 5)),
 (-2, (5, 5)),
 (0, (5, 5))]

In [18]:
vertical_sides

[(0, (-8, -8)),
 (1, (-8, -8)),
 (0, (-6, -6)),
 (1, (-6, -6)),
 (0, (-4, -3)),
 (1, (-4, -3)),
 (0, (-1, -1)),
 (1, (-1, -1)),
 (1, (-8, -8)),
 (2, (-8, -8)),
 (1, (-6, -6)),
 (2, (-6, -6)),
 (1, (-4, -3)),
 (2, (-4, -3)),
 (1, (-1, -1)),
 (2, (-1, -1)),
 (2, (-8, -8)),
 (4, (-8, -8)),
 (2, (-6, -6)),
 (4, (-6, -6)),
 (2, (-4, -3)),
 (4, (-4, -3)),
 (2, (-1, -1)),
 (4, (-1, -1)),
 (4, (-8, -8)),
 (6, (-8, -8)),
 (4, (-6, -6)),
 (6, (-6, -6)),
 (4, (-4, -3)),
 (6, (-4, -3)),
 (4, (-1, -1)),
 (6, (-1, -1))]

In [19]:
len(vertical_sides), len(set(vertical_sides)), len(horizontal_sides), len(set(horizontal_sides))

(32, 20, 16, 10)

In [20]:
vertical_sides = set(sorted(list(set(vertical_sides))))
horizontal_sides = set(sorted(list(set(horizontal_sides))))

In [21]:
vertical_sides, horizontal_sides

({(0, (-8, -8)),
  (0, (-6, -6)),
  (0, (-4, -3)),
  (0, (-1, -1)),
  (1, (-8, -8)),
  (1, (-6, -6)),
  (1, (-4, -3)),
  (1, (-1, -1)),
  (2, (-8, -8)),
  (2, (-6, -6)),
  (2, (-4, -3)),
  (2, (-1, -1)),
  (4, (-8, -8)),
  (4, (-6, -6)),
  (4, (-4, -3)),
  (4, (-1, -1)),
  (6, (-8, -8)),
  (6, (-6, -6)),
  (6, (-4, -3)),
  (6, (-1, -1))},
 {(-9, (3, 3)),
  (-9, (5, 5)),
  (-7, (3, 3)),
  (-7, (5, 5)),
  (-5, (3, 3)),
  (-5, (5, 5)),
  (-2, (3, 3)),
  (-2, (5, 5)),
  (0, (3, 3)),
  (0, (5, 5))})

In [22]:
len(vertex_points), len(set(vertex_points))

(64, 25)

In [23]:
vertex_points = sorted(list(set(vertex_points)))

In [24]:
def oned_intersection(interval1, interval2):
    # boundaries inclusive
    # just checks if two intervals intersect, returns bool
    if interval1[0] > interval2[0]:
        interval1, interval2 = interval2, interval1
    
    return interval1[1] >= interval2[0]

In [25]:
oned_intersection((0, 1), (1, 1)), oned_intersection((1, 1), (1, 1))

(True, True)

In [26]:
horizontal_segments

[(0, (0, 6)),
 (-5, (4, 6)),
 (-7, (4, 6)),
 (-9, (1, 6)),
 (-7, (0, 1)),
 (-5, (0, 2)),
 (-2, (0, 2))]

In [27]:
# now, we go through and check which of these sides and vertices are along the original path
# by intersecting in 1d
vertex_points_along_path = []

for p in vertex_points:
    for h in horizontal_segments:
        if p[1] == h[0] and oned_intersection(h[1], (p[0], p[0])):
            vertex_points_along_path.append(p)
            break
    
    for v in vertical_segments:
        if p[0] == v[0] and oned_intersection(v[1], (p[1], p[1])):
            vertex_points_along_path.append(p)
            break

vertex_points_along_path = set(vertex_points_along_path)

In [28]:
vertex_points_along_path

{(0, -7),
 (0, -5),
 (0, -2),
 (0, 0),
 (1, -9),
 (1, -7),
 (1, -5),
 (1, -2),
 (1, 0),
 (2, -9),
 (2, -5),
 (2, -2),
 (2, 0),
 (4, -9),
 (4, -7),
 (4, -5),
 (4, 0),
 (6, -9),
 (6, -7),
 (6, -5),
 (6, -2),
 (6, 0)}

In [29]:
def check_segment_intersection(s1, s2):
    return s1[0] == s2[0] and oned_intersection(s1[1], s2[1])

In [30]:
horizontal_sides_along_path = set()
vertical_sides_along_path = set()

for side in horizontal_sides:
    for h in horizontal_segments:
        if check_segment_intersection(side, h):
            horizontal_sides_along_path.add(side)
            break

for side in vertical_sides:
    for v in vertical_segments:
        if check_segment_intersection(side, v):
            vertical_sides_along_path.add(side)
            break

In [31]:
horizontal_sides - horizontal_sides_along_path, vertical_sides - vertical_sides_along_path

({(-7, (3, 3)), (-5, (3, 3)), (-2, (3, 3)), (-2, (5, 5))},
 {(0, (-8, -8)),
  (0, (-4, -3)),
  (1, (-6, -6)),
  (1, (-4, -3)),
  (1, (-1, -1)),
  (2, (-8, -8)),
  (2, (-6, -6)),
  (2, (-1, -1)),
  (4, (-8, -8)),
  (4, (-4, -3)),
  (4, (-1, -1)),
  (6, (-6, -6))})

In [32]:
horizontal_sides_along_path

{(-9, (3, 3)),
 (-9, (5, 5)),
 (-7, (5, 5)),
 (-5, (5, 5)),
 (0, (3, 3)),
 (0, (5, 5))}

In [33]:
vertical_sides_along_path

{(0, (-6, -6)),
 (0, (-1, -1)),
 (1, (-8, -8)),
 (2, (-4, -3)),
 (4, (-6, -6)),
 (6, (-8, -8)),
 (6, (-4, -3)),
 (6, (-1, -1))}

In [55]:
def point_in_interior(point, check_vertical=True):
    # if this point lines on any 'natural' gridline, all bets are off.
    # making the assumption that it doesn't.

    # count how many times we pass the boundary going from outside to this point
    # if it's odd, we're inside
    x, y = point

    if check_vertical:
        applicable_vertical_sides = sorted([vs for vs in vertical_sides_along_path if oned_intersection(vs[1], (y, y))])
        print(applicable_vertical_sides)

        # number of vertical sides with x coordinate less than x
        n = sum([1 for vs in applicable_vertical_sides if vs[0] < x])

        # odd parity
        return n % 2 == 1
    else:
        applicable_horizontal_sides = sorted([hs for hs in horizontal_sides_along_path if oned_intersection(hs[1], (x, x))])
        print(applicable_horizontal_sides)

        n = sum([1 for hs in applicable_horizontal_sides if hs[0] < y])

        return n % 2 == 1

In [56]:
point_in_interior((0, -9)), point_in_interior((2, -8))

[]
[(1, (-8, -8)), (6, (-8, -8))]


(False, True)

In [57]:
point_in_interior((6, -6)), point_in_interior((3, -6))

[(0, (-6, -6)), (4, (-6, -6))]
[(0, (-6, -6)), (4, (-6, -6))]


(False, True)

In [58]:
point_in_interior((-11, 1)), point_in_interior((1, 1)), point_in_interior((1, 5))

[]
[]
[]


(False, False, False)

In [59]:
point_in_interior((5, 4))

[]


False

In [60]:
interior_rectangle

[(3, 3, -8, -8),
 (3, 3, -6, -6),
 (3, 3, -4, -3),
 (3, 3, -1, -1),
 (5, 5, -8, -8),
 (5, 5, -6, -6),
 (5, 5, -4, -3),
 (5, 5, -1, -1)]

In [61]:
total_rectangle_interior_area = 0
for (x_start, x_end, y_start, y_end) in interior_rectangle:
    # just check if point from rectangle is interior
    if point_in_interior((x_start, y_start)):
        # print('interior!')
        ia = (1+x_end - x_start) * (1+y_end - y_start)
        print(ia)
        total_rectangle_interior_area += ia
    else:
        print('exterior!')

[(1, (-8, -8)), (6, (-8, -8))]
1
[(0, (-6, -6)), (4, (-6, -6))]
1
[(2, (-4, -3)), (6, (-4, -3))]
2
[(0, (-1, -1)), (6, (-1, -1))]
1
[(1, (-8, -8)), (6, (-8, -8))]
1
[(0, (-6, -6)), (4, (-6, -6))]
exterior!
[(2, (-4, -3)), (6, (-4, -3))]
2
[(0, (-1, -1)), (6, (-1, -1))]
1


In [62]:
total_rectangle_interior_area

9

In [63]:
total_1d_along_boundary = 0

for (y, (x_start, x_end)) in horizontal_sides_along_path:
    total_1d_along_boundary += (x_end + 1 - x_start)

for (x, (y_start, y_end)) in vertical_sides_along_path:
    total_1d_along_boundary += (y_end + 1 - y_start)

In [64]:
horizontal_sides - horizontal_sides_along_path

{(-7, (3, 3)), (-5, (3, 3)), (-2, (3, 3)), (-2, (5, 5))}

In [65]:
vertical_sides - vertical_sides_along_path

{(0, (-8, -8)),
 (0, (-4, -3)),
 (1, (-6, -6)),
 (1, (-4, -3)),
 (1, (-1, -1)),
 (2, (-8, -8)),
 (2, (-6, -6)),
 (2, (-1, -1)),
 (4, (-8, -8)),
 (4, (-4, -3)),
 (4, (-1, -1)),
 (6, (-6, -6))}

In [66]:
total_1d_along_internal = 0

for (c, (start, end)) in horizontal_sides - horizontal_sides_along_path:
    # now we have to check if this side is interior, going perpendicular to its direction
    if point_in_interior((start, c), check_vertical=False):
        print("adding horizontal side", c, start, end)
        total_1d_along_internal += (end + 1 - start)

for (c, (start, end)) in vertical_sides - vertical_sides_along_path:
    if point_in_interior((c, start), check_vertical=True):
        print("adding vertical side", c, start, end)
        total_1d_along_internal += (end + 1 - start)

[(-9, (3, 3)), (0, (3, 3))]
adding horizontal side -2 3 3
[(-9, (3, 3)), (0, (3, 3))]
adding horizontal side -5 3 3
[(-9, (3, 3)), (0, (3, 3))]
adding horizontal side -7 3 3
[(-9, (5, 5)), (-7, (5, 5)), (-5, (5, 5)), (0, (5, 5))]
adding horizontal side -2 5 5
[(0, (-6, -6)), (4, (-6, -6))]
adding vertical side 2 -6 -6
[(0, (-1, -1)), (6, (-1, -1))]
adding vertical side 1 -1 -1
[(0, (-6, -6)), (4, (-6, -6))]
[(0, (-6, -6)), (4, (-6, -6))]
adding vertical side 1 -6 -6
[(2, (-4, -3)), (6, (-4, -3))]
adding vertical side 4 -4 -3
[(2, (-4, -3)), (6, (-4, -3))]
[(1, (-8, -8)), (6, (-8, -8))]
[(1, (-8, -8)), (6, (-8, -8))]
adding vertical side 4 -8 -8
[(0, (-1, -1)), (6, (-1, -1))]
adding vertical side 4 -1 -1
[(1, (-8, -8)), (6, (-8, -8))]
adding vertical side 2 -8 -8
[(0, (-1, -1)), (6, (-1, -1))]
adding vertical side 2 -1 -1
[(2, (-4, -3)), (6, (-4, -3))]


In [67]:
# now for the sides which aren't along the path we need to check if they're interior

In [68]:
total_0d_area = len(vertex_points_along_path)

In [69]:
len(vertex_points)

25

In [70]:
set(vertex_points) - vertex_points_along_path

{(0, -9), (2, -7), (4, -2)}

In [71]:
for p in set(vertex_points) - vertex_points_along_path:
    if point_in_interior(p):
        print("hi")

[]
[]
[]


In [72]:
total_0d_area, total_1d_along_boundary, total_1d_along_internal, total_rectangle_interior_area

(22, 16, 13, 9)

In [73]:
total_0d_area + total_1d_along_boundary

38

In [74]:
total_0d_area+ total_1d_along_boundary+ total_1d_along_internal+ total_rectangle_interior_area

60

In [75]:
# write function which can tell if a point is inside or outside the figure. can just count the number of lines we cross.
# then, for each rectangle in our grid we sample a point and tell if its in or out and add its interior area if its internal to the figure
# and for each interior of each line segment corresponding to a rectangle side we do the same
# and finally for each vertex of the lines we do the same

In [94]:
import math


[0, 5, 8, 9, 10, 15, 17, 18, 19, 20, 25, 28]
[inf, 5, 3, 1, 1, 5, 2, 1, 1, 1, 5, 3]


In [179]:
def num_less_than_chunk_sequential(lst, val):
    # lst = [0, 5, 8, 9, 10, 15, 17, 18, 19, 20, 25, 28] 
    # val = 23

    # [1,2,3,4,5,100], 80 => 1

    if val in lst:
        assert False

    lst = sorted(lst)

    diffs = [b - a for (a, b) in list(zip([-math.inf] + lst, lst))]

    assert len(lst) == len(diffs)

    # print(lst)
    # print(diffs)
    # print(val)
    # print()

    i = next((i for i, x in enumerate(lst) if x > val), 0)

    return sum(1 for d in diffs[:i] if d != 1)

In [180]:
vertex_points_along_path_as_vertical_lines = set((x, (y, y)) for (x, y) in vertex_points_along_path)
vertex_points_along_path_as_horizontal_lines = set((y, (x, x)) for (x, y) in vertex_points_along_path)

In [181]:
def point_in_interior2(point, check_vertical=True):
    # if this point lines on any 'natural' gridline, all bets are off.
    # making the assumption that it doesn't.

    # count how many times we pass the boundary going from outside to this point
    # if it's odd, we're inside
    x, y = point

    if check_vertical:
        applicable_vertical_sides = sorted([vs for vs in vertical_sides_along_path | vertex_points_along_path_as_vertical_lines if oned_intersection(vs[1], (y, y))])
        # print(applicable_vertical_sides)
        n = num_less_than_chunk_sequential([vs[0] for vs in applicable_vertical_sides], x)
    else:
        applicable_horizontal_sides = sorted([hs for hs in horizontal_sides_along_path | vertex_points_along_path_as_horizontal_lines if oned_intersection(hs[1], (x, x))])
        # print(applicable_horizontal_sides)
        n = num_less_than_chunk_sequential([hs[0] for hs in applicable_horizontal_sides], y)

    return n % 2 == 1

In [186]:
other_points_in_c = 0

# also have to check the points which aren't on the path
for p in set(vertex_points) - vertex_points_along_path:
    # print(p)

    r1 = point_in_interior2(p, check_vertical=True)
    r2 = point_in_interior2(p, check_vertical=False)

    assert r1 == r2


    if r1:
        other_points_in_c += 1

other_points_in_c

2